In [1]:
from fastcore.all import *
from fastai.vision.widgets import *
from fastai.vision.all import *

import os
from os.path import join

import PIL
import numpy as np

import matplotlib.pyplot as plt

import torch.nn
import torch
from torch.utils.data import dataloader, DataLoader

from tqdm import tqdm

In [2]:
path = untar_data(URLs.MNIST)
path

Path('C:/Users/0rc44/.fastai/data/mnist_png')

In [7]:
train_path = path/'training'
test_path = path/'testing'

In [5]:
train_labels = list()
train_data = list()
for root,dirs,files in os.walk(train_path):
    for file in files:
        if file.endswith(".png"):
            img = PIL.Image.open(join(root,file))
            train_data.append(np.asarray(img).flatten())
            train_labels.append(parent_label(join(root,file)))

# data[0]

train_labels = np.array(train_labels,dtype=int)
train_data = np.stack(train_data,axis=0)

In [8]:
test_labels = list()
test_data = list()
for root,dirs,files in os.walk(test_path):
    for file in files:
        if file.endswith(".png"):
            img = PIL.Image.open(join(root,file))
            test_data.append(np.asarray(img).flatten())
            test_labels.append(parent_label(join(root,file)))

# data[0]

test_labels = np.array(test_labels,dtype=int)
test_data = np.stack(test_data,axis=0)

In [5]:
train_data.shape

(60000, 784)

In [6]:
train_labels.shape

(60000,)

In [12]:
train_data_n = train_data / 255
test_data_n = test_data / 255

train_y = train_labels
test_y = test_labels

In [13]:
X_train = torch.tensor(train_data_n,dtype=torch.float32)
X_test = torch.tensor(test_data_n,dtype=torch.float32)


y_train = torch.tensor(train_y,dtype=torch.int64)
y_test = torch.tensor(test_y,dtype=torch.int64)

In [14]:
dset = list(zip(X_train,y_train))
dset_valid = list(zip(X_test,y_test))

In [15]:
dl = DataLoader(dset,batch_size=5,shuffle=True)

In [16]:
dl_valid = DataLoader(dset_valid,batch_size = 264)

In [17]:
dls = DataLoaders(dl,dl_valid)

In [19]:
X_train

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [20]:
X_test

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [21]:
y_train

tensor([0, 0, 0,  ..., 9, 9, 9])

In [22]:
y_test

tensor([0, 0, 0,  ..., 9, 9, 9])

In [23]:
n_inputs = 28*28
n_outputs = len(np.unique(train_labels))

seq = torch.nn.Sequential(

     torch.nn.Linear(in_features=n_inputs,out_features=128)
    ,torch.nn.ReLU()
    ,torch.nn.Linear(in_features=128,out_features=64)
    ,torch.nn.ReLU()
    ,torch.nn.Linear(in_features=64,out_features=n_outputs)
    ,torch.nn.Softmax(dim=1)


)


learn = Learner(dls,seq,loss_func = torch.nn.NLLLoss())

In [24]:
learn.fit(10,lr=1e-3)

epoch     train_loss  valid_loss  time    
0         -0.926422   -0.927357   00:38                                                                              
1         -0.954916   -0.952713   00:47                                                                              
2         -0.962579   -0.963556   00:55                                                                              
3         -0.963544   -0.956571   00:52                                                                              
4         -0.959253   -0.957820   00:53                                                                              
5         -0.970195   -0.967904   00:51                                                                              
6         -0.944038   -0.965109   00:54                                                                              
7         -0.975436   -0.969567   00:50                                                                              
8         -0.

In [28]:
(learn.model(X_train).argmax(axis=1) == y_train).float().mean()

tensor(0.9785)